This script describes how to load the condensed json versions of the Gaussian process (GP) interpolation models described in [link to paper]. The models are originally made using the `scikit-learn` Gaussian process regressor module (https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html). The json versions are much more storage-efficient and simply load the hyperparameters of the trained models into a newly instantiated GP regressor object.

The models are first organized into different families by viewing angle. Each viewing angle directory contains time-step subdirectories which host the optimized hyperparameters for interpolation at the given time step. The final light curve product is produced by "stitching together" the separate predictions produced from the interpolators at each separate time step. 

The example below shows how to load the models for the pole-on viewing angle bin (between 0 and 15.64 degrees) and create a prediction for an off-sample parameter combination. The final light curve is stitched together from the 191 separate time step predictions. The light curve output can be compared to the bottom half of Figure 4 in [link to paper].

This script can be generally applied to all viewing angle model families. It is also capable of generating a light curve using only a subset of the time steps in the respective viewing angle directory.

In [1]:
# Jupyter notebook describing how to load and make predictions using json models

import glob
import numpy as np
import save_sklearn_gp as ssg
import matplotlib.pyplot as plt
from scipy.linalg import cholesky, solve_triangular, cho_solve

# making a size (9, 5) array with each row having 5 input parameters (md, vd, mw, vw, wav) for all wavelength bands

times = np.logspace(np.log10(0.125), np.log10(7.6608262), 191)

inputs = np.array([0.097050, 0.197642, 0.083748, 0.297978]) # parameters used for off-sample prediction in paper
wavs = np.array([476., 621., 754., 900., 1020., 1220., 1630., 2190., 4493.]).reshape(9, 1)
inputs = np.tile(inputs, wavs.shape[0]).reshape(-1, 4)
inputs = np.hstack((inputs, wavs))

# loading all the models from the family trained using a pole-on viewing angle

files = glob.glob('theta00deg/*')
files.sort() # sorting is necessary! otherwise time will be out of order

# loop through all the individual time steps, making a prediction at each

for file in files:
	fname = file+'/model'
	model = ssg.load_gp(fname) # loading Gaussian Process from hyperparameters saved in .json format
	K = model.kernel_(model.X_train_) # setting up the kernel
	K[np.diag_indices_from(K)] += model.alpha
	model.L_ = cholesky(K, lower=True) # recalculating L matrix since this is what makes the pickled models bulky
	model._K_inv = None # has to be set to None so the GP knows to re-calculate matrices used for uncertainty
	pred, err = model.predict(inputs, return_std=True) # returns prediction and uncertainty
	try:
		lc = np.append(lc, pred[None, :], axis=0)
		errs = np.append(errs, err[None, :], axis=0)
	except NameError:
		lc = pred[None, :] # if first time point, initializes the light curve array
		errs = err[None, :] # same as above, but for errors
for band in range(lc.shape[1]):
	plt.plot(times, lc[:, band])
	plt.fill_between(times, lc[:, band]-errs[:, band], lc[:, band]+errs[:, band], color='red', alpha=0.3)
plt.xscale('log')
plt.savefig('pred_compact.png')

